In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Module_Four_Python_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("Doupe")

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    # Hidden div for potential interactivity
    html.Div(id='hidden-div', style={'display':'none'}),
      
    # Header and Logo
    html.Div ([
        html.Div([
            html.A(
                href='https://www.snhu.edu', # Link to client's home page
                target='_blank', # Open the link in a new tab
                children=[
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={'height': '80px', 'width': 'auto', 'float': 'left', 'margin-right': '20px'}
                    )                    
                ]
            )
        ], style={'display': 'inline-block', 'verticalAlign': 'middle'}),
        
        html.Div([
            html.B(html.H1('CS-340 Dashboard', style={'margin': '0', 'padding': '0'})),
            html.P(
                "Created by: Tyler Doupe",
                style={'fontSize': '16px', 'margin': '5px 0 0 0', 'color': 'gray'}
            )
        ], style={'display': 'inline-block', 'verticalAlign': 'middle', 'margin-left': '20px'})
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '10px'}),
        
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Label('Rescue Types:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            inline=True
        )
    ]),
    
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         style_table={'overFlowX': 'auto'},
                         style_cell={
                             'textAlign': 'left',
                             'padding': '10px',
                             'fontFamily': 'Arial',
                             'fontSize': '14px',
                         },
                         style_header={
                             'backgroundColor': 'rgb(210, 210, 210)',
                             'fontWeight': 'bold'
                         },
                         row_selectable= 'single',
                         selected_rows=[0], # Sets the first row as selected by default
                         page_size=10, #Limits the rows per page to 10
                         filter_action='native',
                         sort_action='native',
                         sort_mode='multi'
                     ),
                     html.Br(),
                     html.Hr(),
                     html.Div(
                             id='map-id',
                             className='col s12 m6',
                         ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id','data'),
    Input('filter-type', 'value')
)

def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
    # Create a query dictionary for filtering
    
    # Define the filter query
    if filter_type == 'Water Rescue':
        query = {
            'breed': {'$in': ['Labrador Retrienver Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': {'$regex': 'Intact Female', '$options': 'i'},
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            'breed': {'$in': ['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': {'$regex': 'Intact Male', '$options': 'i'},
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }    
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': {'$regex': 'Intact Male', '$options': 'i'},
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }     
    else: # Reset
        query = {}
    
    # Retrieve filtered data
    filtered_data = db.read(query)
    if filtered_data:
        df_filtered = pd.DataFrame.from_records(filtered_data)
        df_filtered.drop(columns=['_id'], inplace=True)
        return df_filtered.to_dict('records')
    return []

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('filter-type', 'value')])
def update_graphs(viewData, filter_type):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    # Return pie chart of animal distribution
    if viewData is None or len(viewData) == 0:
        return html.Div ("No Data Available")
    
    # Convert the view data into a data frame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a title for the pie chart based on the selected filter type
    chart_title = "Preferred Animals for " + filter_type if filter_type != "Reset" else "All Animals"
       
    # Check if the 'breed' column exists
    if 'breed' in dff.columns:
        if filter_type == "Water Rescue":
            relevant_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        elif filter_type == "Mountain or Wilderness Rescue":
            relevant_breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        elif filter_type == "Disaster or Individual Tracking":
            relevant_breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        else:
            relevant_breeds = dff['breed'].unique()  # Reset mode shows all breeds

        # Filter DataFrame to include only relevant breeds
        dff = dff[dff['breed'].isin(relevant_breeds)]
        
        # Return the updated pie chart
        return [
            dcc.Graph(
                figure=px.pie(
                    dff,
                    names='breed',
                    title=chart_title,
                    hole=0.4,  # Add a donut style for better readability
                    color_discrete_sequence=px.colors.qualitative.Pastel  # Optional: better color palette
                )
            )
        ]
    else:
        return html.Div("No 'breed' column available in the data")
            
        
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Initializing with user=aacuser, passwd=SNHU
Dash app running on http://127.0.0.1:10472/
